<a href="https://colab.research.google.com/github/tarabelo/PIAC-apuntes/blob/main/Algoritmos%20cu%C3%A1nticos%20cl%C3%A1sicos%20(primera%20parte).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para simular y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
    # Usamos el simulador de vector de estado
    # https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html
    sim = AerSimulator(method='statevector')
    qc_obj = transpile(qcirc, sim)
    result = sim.run(qc_obj).result()
    estado = result.get_statevector()
    display(estado.draw('latex', prefix=etiqueta))
    if bloch:
      display(estado.draw('bloch'))

# Funcion para obtener y mostrar la matriz unitaria
def obten_unitaria(qcirc, etiqueta):
    # Usamos el simulador de matriz unitaria
    sim_u = AerSimulator(method='unitary')
    qc_obj = transpile(qcirc, sim_u)
    result = sim_u.run(qcirc).result()
    unitary = result.get_unitary(qcirc)
    display(unitary.draw('latex', prefix=etiqueta))

# Algoritmos cuánticos "clásicos"

Algoritmos que presentan una ventaja real sobre sus homólogos clásicos.

Algunas características:
 - Introducidos en los años 90 del siglo XX
 - Suponen que tenemos computadores cuánticos ideales
 - Algunos de ellos carecen de utilidad práctica, mientras que otros suelen formar parte de algoritmos más complejos.

### Contenidos

1. [Paralelismo cuántico](#paralelismo)
1. [Oráculos cuánticos](#oraculos)
2. [Algoritmo de Deutsch](#deutsch)
3. [Algoritmo de Deutsch-Jozsa](#dj)
4. [Algoritmo de Bernstein-Vazirani](#bv)
4. [Algoritmo de periodicidad de Simon](#simon)
5. [Algoritmo de búsqueda de Grover: amplificación de amplitud](#grover)

# Paralelismo cuántico <a name="paralelismo"></a>

Permite evaluar una función $f(x)$ para múltiples valores de $x$ simultáneamente.

Sea $f(x): \{0,1\}\rightarrow\{0,1\}$. Para poder usar esa función en un sistema cuántico tenemos que obtener una unitaria $U_f$ que la implemente. Una forma simple es:

<center><img src="https://drive.google.com/uc?export=view&id=1wE_Ts7MoneY2S4p11SqYHT4o0_SI7vEd" alt="Puerta reversible" width="300"  /></center>

Al cúbit auxiliar $|y\rangle$ se le denomina *ancilla*


Es fácil ver que $U_f$ es su propia inversa:

<center><img src="https://drive.google.com/uc?export=view&id=1pHBMmR3su74ziSuXJK0p9zr1sAGejti_" alt="Puerta reversible demo" width="600"  /></center>


Para un sistema de dos cúbits $|yx\rangle=|y\rangle\otimes|x\rangle$ (siguiendo el orden de Qiskit) tenemos:

$$
U_f|yx\rangle = |y\oplus f(x)\rangle\otimes|x\rangle
$$

Si hacemos $|x\rangle = \tfrac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$ e $|y\rangle=|0\rangle$:

$$
\begin{aligned}
U_f\left(|0\rangle\otimes|\tfrac{1}{\sqrt{2}}(|0\rangle+|1\rangle)\rangle\right) &
= U_f\left(|\tfrac{1}{\sqrt{2}}(|00\rangle+|01\rangle)\right)
= \tfrac{1}{\sqrt{2}}\left(U_f|00\rangle+U_f|01\rangle\right) \\
& = \tfrac{1}{\sqrt{2}}\left(|0\oplus f(0)\rangle\otimes|0\rangle+|0\oplus f(1)\rangle\otimes|1\rangle\right)
= \tfrac{1}{\sqrt{2}}\left(|f(0)\rangle\otimes|0\rangle+|f(1)\rangle\otimes|1\rangle\right)
\end{aligned}
$$

Una única aplicación de $U_f$ obtiene a la vez $f(0)$ y $f(1)$.

Este resultado se puede generalizar a un estado de $n$ cubits $|\psi\rangle = \tfrac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |i\rangle$:

$$
U_f\left(|0\rangle\otimes|\psi\rangle\right) = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |f(i)\rangle\otimes|i\rangle
$$

Se evalúan simultáneamente todos los $f(x), \forall x=0\ldots 2^{n-1}$!

&nbsp;

<details><summary>¿Sirve esto para algo?</summary>
Realmente, este paralelismo no sirve de mucho, ya que al medir solo vamos a obtener un valor de  $|i,f(i)\rangle$ (y, además, no sabemos cuál va a ser!).
</details>



---



---



---



# Oráculos cuánticos <a name="oraculos"></a>

# Oráculos <a name="oraculo"></a>

Función de "caja negra" de la que solo podemos conocer su salida dada una entrada.

Los primeros algoritmos que veremos permiten extraer propiedades de un oráculo.


#### Ejemplo:

Sea una función $f(x): \{0,1\}\rightarrow\{0,1\}$. Solo hay 4 versiones de esta función:

<center><img src="https://drive.google.com/uc?export=view&id=1hLNmVOxHoBaCkN6zYnEV3YP10aXIhwJk" alt="Oráculo" width="600"  /></center>


Estas funciones pueden expresarse como matrices:

$$
f_0 = \begin{bmatrix}1 & 0\\0 & 1\end{bmatrix}\qquad
f_1 = \begin{bmatrix}0 & 1\\1 & 0\end{bmatrix}\qquad
f_2 = \begin{bmatrix}1 & 1\\0 & 0\end{bmatrix}\qquad
f_3 = \begin{bmatrix}0 & 0\\1 & 1\end{bmatrix}
$$

Podemos convertir esas matrices en unitarias $U_{f_i}$ para usar en el circuito anterior.

Por ejemplo, para $f_0$, podemos obtener la matriz correspondiente a $U_f$ a partir de la siguiente tabla:

<center><img src="https://drive.google.com/uc?export=view&id=1q0VWpLV2AvreY43Jms_UMfs0Ahb8tRIM" alt="Tabla f0" width="400"  /></center>


Por tanto la matriz $U_{f_0}$ es:

$$
U_{f_0} =
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\end{bmatrix}
$$

### Ejercicio 1

Obtén las matrices para las otras tres funciones.



---



La matriz $U_{f_0}$ corresponde a una puerta CNOT:

In [ ]:
x = QuantumRegister(1, name="|x\\rangle")
y = QuantumRegister(1, name="|y\\rangle")
qc = QuantumCircuit(x,y)

# Aplica CNOT, primer parámetro target, segundo control
qc.cx(x,y)
qc.save_unitary()
qc.draw('mpl')

In [ ]:
obten_unitaria(qc, "U_{f_0} = ")

Qiskit permite definir este circuito como una puerta:

In [ ]:
# Oráculo para f0
def f0_oraculo():
    """
    Define un oraculo para la funcion f0
        return: circuito de 2 cúbit en forma de puerta
    """
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)
    # Aplica CNOT, primer parámetro target, segundo control
    qc.cx(x,y)


    oraculo = qc.to_gate()
    oraculo.name = "$U_{f_0}$"
    return(oraculo)

### Demostración del paralelismo cuántico

Podemos ver como funciona el paralelismo cuántico haciendo un circuito en Qiskit con el oraculo asociado a $f_0$.

In [ ]:
x = QuantumRegister(1, name="|x\\rangle")
y = QuantumRegister(1, name="|y\\rangle")
qc = QuantumCircuit(x,y)
estado_mas = Statevector([1/sqrt(2), 1/sqrt(2)])
qc.initialize(estado_mas,0)

# Añadimos el oráculo
qc = qc.compose(f0_oraculo())
qc.save_statevector()
qc.draw('mpl')

In [ ]:
obten_estado(qc)

Se trata del estado:
$$
|\psi\rangle = \tfrac{1}{\sqrt{2}}(|00\rangle+|11\rangle) = \tfrac{1}{\sqrt{2}}(|f_0(0),0\rangle+|f_1(1),1\rangle)
$$

### Ejercicio 2

Crea funciones oráculo en Qiskit para las otras 3 matrices y prueba el paralelismo cuantico en los 3 casos.

## Complejidad de consultas

En un algoritmo que hace uso de un oráculo, se denomina complejidad de consultas (*query complexity*) al número medio de *consultas* al oráculo que se necesitan para resolver el problema.

La complejidad se expresa a través de la *notación asintótica*, principalmente la notación *Big-O*.

Así, si el número $n$ de accesos al oráculo crece exponencialmente, la complejidad será $\mathcal{O}\left(2^n\right)$ y si lo hace linealmente $\mathcal{O}\left(n\right)$.

Los algoritmos cuánticos que veremos a continuación tienen una menor complejidad de consultas que sus equivalentes clásicos.

-------------------------------
-------------------------------
-------------------------------

# Algoritmo de Deutsch <a name="deutsch"></a>

Las funciones $f(x): \{0,1\}\rightarrow\{0,1\}$ se denominan:

- Constantes: si $f(0)=f(1)$
- Balanceadas: si $f(0)\ne f(1)$

De las 4 funciones anteriores, $f_0$ y $f_1$ son balanceadas mientras que $f_2$ y $f_3$ son constantes.

Dada una función $f(x): \{0,1\}\rightarrow\{0,1\}$ desconocida, un algoritmo clásico necesita 2 llamadas a $f$ para determinar si es constante o balanceada.

El **algoritmo cuántico de Deutsch** puede determinarlo con una sola llamada a $f$.

Partimos del circuito siguiente:

<center><img src="https://drive.google.com/uc?export=view&id=1npbfKEwOqZA1EOUl8JeoytuQ7e0b7v1C" alt="Deutsch" width="800"  /></center>

Estado después de la puerta X:
$$
|\psi_0\rangle = |10\rangle
$$

Estado despues de las H:
$$
|\psi_1\rangle = |-+\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}\otimes \frac{|0\rangle + |1\rangle}{\sqrt{2}} = |y\rangle \otimes|x\rangle
$$

Y se tiene que:

$$
y\oplus f(x) = \frac{|0\rangle - |1\rangle}{\sqrt{2}}\oplus f(x) = \frac{|0\oplus f(x)\rangle - |1\oplus f(x)\rangle}{\sqrt{2}} =
\begin{cases}
\frac{|0\rangle - |1\rangle}{\sqrt{2}}, & \text{si } f(x) = 0\\
\frac{|1\rangle - |0\rangle}{\sqrt{2}}, & \text{si } f(x) = 1
\end{cases}
$$

por lo que podemos escribir:
$$
y\oplus f(x) = (-1)^{f(x)}\frac{|0\rangle - |1\rangle}{\sqrt{2}}
$$

De aquí se tiene que:

$$
|\psi_2\rangle = \frac{1}{\sqrt{2}}(-1)^{f(x)} (|0\rangle - |1\rangle)\otimes |x\rangle =
\frac{1}{\sqrt{2}} (|0\rangle - |1\rangle)\otimes (-1)^{f(x)}|x\rangle
$$
Y podemos escribir:
$$
(-1)^{f(x)}|x\rangle = |(-1)^{f(x)}x\rangle =
\frac{(-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle}{\sqrt{2}}
$$

Por lo tanto, el estado $|\psi_2\rangle$ queda:
$$
|\psi_2\rangle =
\frac{|0\rangle - |1\rangle}{\sqrt{2}}\otimes
\frac{(-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle}{\sqrt{2}} =
\begin{cases}
\pm\left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right] \otimes \left[\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right], & \text{si } f(0) = f(1)\\\mbox{}\\
\pm\left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right] \otimes  \left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right], & \text{si } f(0) \neq f(1)
\end{cases}
$$

Al aplicar la última `H`:

$$
|\psi_3\rangle =
\begin{cases}
\pm\frac{|0\rangle - |1\rangle}{\sqrt{2}} \otimes |0\rangle, & \text{si } f(0) = f(1)\\\mbox{}\\
\pm\frac{|0\rangle - |1\rangle}{\sqrt{2}} \otimes |1\rangle, & \text{si } f(0) \neq f(1)
\end{cases}
$$

Midiendo el cúbit de arriba vamos a obtener, con total certeza, un $0$ si $f(0)=f(1)$ o un $1$ si $f(0)\ne f(1)$, con una sola evaluacion de $f(x)$.


### Ejercicio 3

Implementa un circuito en Qiskit para probar el algoritmo de Deutsch con los oráculos definidos para las funciones $f_{0-3}$. Comprueba que es capaz de determinar si la función es constante o balanceada.

In [ ]:
# Empieza creando un circuito a partir de un oráculo con un bit de medida
def deutsch_qc(oraculo):
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    meas = ClassicalRegister(1, name="Medida")
    qc = QuantumCircuit(x,y,meas)

    # TODO: Añade las puertas iniciales


    # TODO: Añade el oraculo
    qc = qc.compose()

    # TODO: Añade la última puerta H

    # TODO: Añade la medida

    # Devuelve el circuito
    return qc

In [ ]:
# Probamos con f0
qc = deutsch_qc(f0_oraculo)
qc.draw('mpl')

In [ ]:
# Hacemos la simulacion
sim = AerSimulator(method='automatic')
qc_obj = transpile(qc, sim)
result = sim.run(qc_obj, shots=1024).result()
plot_histogram(result.get_counts())

In [ ]:
# TODO: Probar con las otras 3 funciones
sim = AerSimulator(method='automatic')
for f in [f1_oraculo, f2_oraculo, f3_oraculo]:
    qc = deutsch_qc(f)
    # hacemos la simulacion
    qc_obj = transpile(qc, sim)
    result = sim.run(qc_obj, shots=1024).result()
    display(plot_histogram(result.get_counts()))



---



---



---



# Algoritmo de Deutsch-Jozsa <a name="dj"></a>

Es una generalización a $n$ bits del anterior. Dada una función (oráculo) $f(\{x_{n-1},x_{n-2},\ldots,x_1,x_{0}\}) \rightarrow \{0,1\}$, que es, o bien, _balanceada_, o bien, _constante_, determinar el tipo:

- Función constante: la salida de $f(x)$ es, o bien, 0, o bien, 1 $\forall x$
- Función balanceada: devuelve 0 para la mitad de las entradas y 1 para la otra mitad

**Solución clásica**

Un algoritmo clásico necesita efectuar en el mejor caso $2$ y en el peor $2^{n-1}+1$ evaluaciones de $f(x)$, por lo que su complejidad es exponencial $\mathcal{O}(2^n)$.

**Solución cuántica**

El algoritmo cuántico ([D. Deutsch and R. Jozsa, 1992](https://doi.org/10.1098%2Frspa.1992.0167)) solo necesita una evaluación del oráculo $f(x)$ (ganancia exponencial).

El circuito usado es:

<center><img src="https://drive.google.com/uc?export=view&id=1SRf5wmYaN4NmLgjT7MXm0v4XsUEIASQW" alt="Deutsch-Jozsa" width="800"  /></center>

Pasos:

<ol>
<li>
Estado inicial:
        

$$\vert \psi_0 \rangle = \vert 1\rangle \vert0\rangle^{\otimes n} $$
</li>
    
<li>
Después de aplicar las primeras puertas H:
$$\vert \psi_1 \rangle = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} \left(|0\rangle - |1 \rangle \right)\vert x\rangle $$
</li>
<p>
<li>
Aplicamos $U_f$ a $\vert y\rangle \vert x\rangle$ para obtener $\vert y \oplus f(x)\rangle\vert x\rangle$, recordando que $y\oplus f(x) = \tfrac{1}{\sqrt{2}} (-1)^{f(x)}(|0\rangle - |1\rangle)$:

$$\vert\psi_2\rangle  = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}(|0\rangle - |1\rangle)|x\rangle=\frac{(|0\rangle - |1\rangle)}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}|x\rangle$$
          </li>
<p>
<li>Ahora aplicamos una H a cada cúbit en  $|x\rangle$ (recordemos que $H^{\otimes n}|x\rangle = \tfrac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle)$. Si ya no consideramos el cúbit de abajo, el estado es:<p>
$$
\begin{aligned}
    \lvert \psi_3 \rangle
        & = \frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)}
            \left[ \sum_{i=0}^{2^n-1}(-1)^{x \cdot i}
            \vert i \rangle \right] \\
        & = \frac{1}{2^n}\sum_{i=0}^{2^n-1}
            \left[ \sum_{x=0}^{2^n-1}(-1)^{f(x)}(-1)^{x \cdot i} \right]
            \vert i \rangle
\end{aligned}
$$
       
con $x \cdot i = x_{n-1}i_{n-1}\oplus x_{n-2}i_{n-2}\oplus \ldots \oplus x_1i_1 \oplus x_0i_0$.
   </li>

   <li>Se realiza la medida. La probabilidad de obtener todo 0 ($\vert i \rangle=  \vert 0 \rangle ^{\otimes n}$) es:
$$
       \left| \frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)} \right|^2 =
       \begin{cases}
       1, \text{si } f \text{ es constante}\\
       0, \text{si } f \text{ es balanceada}
       \end{cases}
$$
   </li>
<p>
</ol>
En conclusión, si a la salida obtenemos todo 0, $f$ es constante. Si obtenemos cualquier otro valor $f$ es balanceada.

<p></p>

<details>
    <summary>Demostración de que $H^{\otimes n}|x\rangle = \tfrac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle$</summary>

El uso de puertas `H` permite poner los n-cúbits en un estado de superposición.

El estado de _superposición completa_ es:

$$
|+\rangle^{\otimes n} = H^{\otimes n}|0\rangle^{\otimes n}
$$

$H^{\otimes n}$ se conoce como puerta _Walsh-Hadamard_.

Ejemplo para 4 cúbits:

$$
H^{\otimes 4}|1010\rangle = H|1\rangle\otimes H|0\rangle\otimes H|1\rangle\otimes H|0\rangle = \\
\frac{1}{4}\left[(|0\rangle - |1\rangle)\otimes (|0\rangle + |1\rangle) \otimes (|0\rangle - |1\rangle) \otimes (|0\rangle + |1\rangle)\right] = \frac{1}{4}\left[\\
|0000\rangle+|0001\rangle-|0010\rangle-|0011\rangle+\\
|0100\rangle+|0101\rangle-|0110\rangle-|0111\rangle-\\
|1000\rangle-|1001\rangle+|1010\rangle+|1011\rangle-\\
|1100\rangle-|1101\rangle+|1110\rangle+|1111\rangle\phantom{-}\right]
$$

En este ejemplo, el signo negativo aparece en los estados $|x_3x_2x_1x_0\rangle$ para los que se verifica que:
    
$$
(x_3x_2x_1x_0)\cdot(1010) = x_3\cdot 1\oplus x_2\cdot 0 \oplus x_1\cdot 1 \oplus x_0\cdot 0 = x_3\cdot x_1 = 1
$$

que son los estados para los que $x_3 \ne x_1$.
    
En general, para un estado $|x\rangle = |x_{n-1}x_{n-2}\ldots x_0\rangle$ de n-cúbits, se puede escribir:

$$
H^{\otimes n}|x\rangle = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle
$$

siendo $x\cdot i = x_{n-1}i_{n-1}\oplus x_{n-2}i_{n-2}\oplus \ldots \oplus x_0i_0$

</details>



**Ejemplo**
Supongamos $n=2$. Las probabilidades asociadas a cada salida son:

$$
p(0) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 0} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{0} + (-1)^{f(2)}(-1)^{0} + (-1)^{f(3)}(-1)^{0}\right]  \right|^2
$$
$$
p(1) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 1} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{1} + (-1)^{f(2)}(-1)^{0} + (-1)^{f(3)}(-1)^{1}\right]  \right|^2
$$
$$
p(2) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 2} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{0} + (-1)^{f(2)}(-1)^{1} + (-1)^{f(3)}(-1)^{1}\right]  \right|^2
$$
$$
p(3) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 3} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{1} + (-1)^{f(2)}(-1)^{1} + (-1)^{f(3)}(-1)^{0}\right]  \right|^2
$$

Es fácil ver que si $f$ es constante, $p(0)=1$ y $p(1)=p(2)=p(3)=0$, ya que se anulan los términos. Por contra, si $f$ es balanceada $p(0)=0$.

### Ejercicio 4

Determina qué valor se va a medir si $f(0) = f(1) = 0$ y $f(2) = f(3) = 1$.

-------------------------------
-------------------------------
-------------------------------

# Algoritmo de Bernstein-Vazirani <a name="bv"></a>

Es una versión del anterior en la que el oráculo $f(\{x_{n-1},x_{n-2},\ldots,x_{0}\}) \rightarrow \{0,1\}$ es de la forma $f_s(x) = s\cdot x = s_{n-1}x_{n-1} \oplus s_{n-2}x_{n-2} \oplus \ldots \oplus s_{0}x_{0}$, con $s_i\in\{0,1\}$, donde $s$ es desconocido. El objetivo es encontrar el valor de $s$.

**Solución clásica**

Para obtener $s$, un algoritmo clásico tiene que hacer $n$ invocaciones de $f_s(x)$ con los valores $x=2^i$ con $i \in \{0,\ldots,n-1\}$.

$$
s_0 = f_s(000\ldots 01)\\
s_1 = f_s(000\ldots 10)\\
\cdots\\
s_{n-1} = f_s(100\ldots 00)
$$

**Solución cuántica**

El algoritmo cuántico ([E. Bernstein y U. Vazirani, 1997](https://doi.org/10.1137/S0097539796300921)) necesita 1 sola invocación del oráculo.

El circuito es idéntico al de Deutsch-Jozsa, solo cambia la función oráculo.

Al igual que antes tenemos:

$$            
\lvert \psi_3 \rangle  = \frac{1}{2^n}\sum_{i=0}^{2^n-1}
                    \left[ \sum_{x=0}^{2^n-1}(-1)^{f_s(x)}(-1)^{x \cdot i} \right]
                    \vert i \rangle =
                    \frac{1}{2^n}\sum_{i=0}^{2^n-1}
                    \left[ \sum_{x=0}^{2^n-1}(-1)^{s\cdot x}(-1)^{x \cdot i} \right]
                    \vert i \rangle =
                    \sum_{i=0}^{2^n-1}
                    \left[ \frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{s\cdot x}(-1)^{x \cdot i} \right]
$$

El sumatorio sobre $x$ se puede escribir:

$$
\frac{1}{2^n} \sum_{x=0}^{2^n-1}(-1)^{s\cdot x}(-1)^{x \cdot i} = \frac{1}{2^n} \sum_{x=0}^{2^n-1}(-1)^{(s\cdot x) \oplus (x \cdot i)} = \frac{1}{2^n} \sum_{x=0}^{2^n-1}(-1)^{(s \oplus i)\cdot x}
=
       \begin{cases}
       1, \text{si } s \oplus i = 0\\
       0, \text{si } s \oplus i = 1
       \end{cases}
$$                    

Y $s \oplus i = 0 \Rightarrow s = i$, es decir, que la amplitud no nula es, precisamente, $|s\rangle$.

**Ejemplo**: supongamos $n=2$. Aplicando el algoritmo tenemos en el paso 2:

$$
\lvert \psi_2 \rangle  
    = \frac{|0\rangle - |1\rangle}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}|x\rangle
    = \frac{|0\rangle - |1\rangle}{2\sqrt{2}}
                 \left[
                 (-1)^{s\cdot 00}|00\rangle +
                 (-1)^{s\cdot 01}|01\rangle +
                 (-1)^{s\cdot 10}|10\rangle +
                 (-1)^{s\cdot 11}|11\rangle\right]
$$

Si suponemos $s=11$, el estado será:

$$
\lvert \psi_2 \rangle =   \frac{|0\rangle - |1\rangle}{2\sqrt{2}}
                 \left[
                 (-1)^{11\cdot 00}|00\rangle +
                 (-1)^{11\cdot 01}|01\rangle +
                 (-1)^{11\cdot 10}|10\rangle +
                 (-1)^{11\cdot 11}|11\rangle\right] =
                 \frac{1}{2}(|00\rangle - |01\rangle - |10\rangle + |11\rangle)
$$

Y al aplicar las `H`:

$$
\lvert \psi_3 \rangle =   \frac{1}{4}\left[
(|0\rangle+|1\rangle)\otimes(|0\rangle+|1\rangle) -
(|0\rangle+|1\rangle)\otimes(|0\rangle-|1\rangle) -
(|0\rangle-|1\rangle)\otimes(|0\rangle+|1\rangle) +
(|0\rangle-|1\rangle)\otimes(|0\rangle-|1\rangle)
\right] = \\
= \frac{1}{4}\left[
|00\rangle + |01\rangle + |10\rangle + |11\rangle - |00\rangle + |01\rangle - |10\rangle + |11\rangle  
- |00\rangle - |01\rangle + |10\rangle + |11\rangle + |00\rangle - |01\rangle - |10\rangle + |11\rangle
\right] = \\
= |11\rangle = |s\rangle
$$

### Ejercicio 5

Implementa un oráculo para el algoritmo de Bernstein-Vazirani con $n$ bits y $s$ aleatoria.

In [ ]:
# Obtenemos s como una lista de valores binarios aleatoria
def get_s(n):
    s = list()
    for q in range(n):
        if np.random.randint(2) == 1:
            s.append(1)
        else:
            s.append(0)
    return(s)

In [ ]:
def bv_oraculo(n):
    """
    Devuelve un oraculo para el algoritmo de Bernstein-Vazirani

        n: Número de cúbits de entrada
        return: circuito de n+1 cúbit en forma de puerta
    """
    x = QuantumRegister(n, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)
    s = get_s(n)

    # El oráculo puede implementarse con puertas CNOT controladas
    # por los cúbits para los que el bit en s vale 1

    sinv = s[::-1] # Invertimos s para recorrerlo del bit menos al más significativo

    ...................................

    # Salida
    oraculo = qc.to_gate()
    oraculo.name = "$U_{f}$"
    return oraculo

------------------------
------------------------
------------------------

# Algoritmo de periodicidad de Simon <a name="simon"></a>

En este caso, el oráculo es una función $f_c:\{0,1\}^n \rightarrow \{0,1\}^n$ que depende de un número binario desconocido $c=c_{n-1}c_{n-2}\ldots c_0 \ne 0$, con $c_i \in \{0,1\}$, tal que, para todos los valores $x,y \in \{0,1\}^n$ se tiene que:
$$
f_c(x) = f_c(y) \Leftrightarrow y=x\oplus c \text{   (o que }x\oplus y =c\text{)}
$$

Es decir, los valores de $f_c$ se repiten con un cierto patrón dado por $c$, que se denomina período. Por ejemplo, supongamos $n=3$ y $c=5\equiv 101$:

$$
\hphantom{00} x\hphantom{0 \oplus 1} c \hphantom{01 = 1} y\hphantom{01 \Rightarrow f_c(0) = f_c(5)}\\
000 \oplus 101 = 101 \Rightarrow f_c(0) = f_c(5)\\
001 \oplus 101 = 100 \Rightarrow f_c(1) = f_c(4)\\
010 \oplus 101 = 111 \Rightarrow f_c(2) = f_c(7)\\
011 \oplus 101 = 110 \Rightarrow f_c(3) = f_c(6)\\
100 \oplus 101 = 001 \Rightarrow f_c(4) = f_c(1)\\
101 \oplus 101 = 000 \Rightarrow f_c(5) = f_c(0)\\
110 \oplus 101 = 011 \Rightarrow f_c(6) = f_c(3)\\
111 \oplus 101 = 010 \Rightarrow f_c(7) = f_c(2)
$$

**Solución clásica**

Un algoritmo clásico necesita evaluar la función hasta encontrar dos valores $x$ e $y$ tales que $f_c(x) = f_c(y)$. Dados esos valores, se tiene que $c=x\oplus y$.

En el peor de los casos tenemos que buscar en la mitad de los valores, efectuando $2^{n-1}+1$ evaluaciones de $f(x)$.

**Solución cuántica**

El algoritmo cuántico ([D. R. Simon, 1997](https://doi.org/10.1137/S0097539796298637)) solo necesita $n$ evaluaciones.

El circuito que se utiliza es similar al que venimos usando pero con $n$ bits de ancilla:

<center><img src="https://drive.google.com/uc?export=view&id=1x8fmuY7WjcTh9iIrjY3-Fhp95pT_IbRf" alt="Simon" width="800"  /></center>



Pasos:

<ol>
   <li>
        Estado inicial:
        

$$\vert \psi_0 \rangle = \vert0\rangle^{\otimes n} \vert 0\rangle^{\otimes n}$$


   </li>
    
<li>Después de aplicar las primeras puertas H:
$$\vert \psi_1 \rangle = \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|0\rangle \vert x\rangle$$
</li>
   <p>
<li>Aplicamos $U_f$ a $\vert y\rangle\vert x\rangle$ para obtener $\vert y \oplus f(x)\rangle\vert x\rangle$:
$$
\lvert \psi_2 \rangle  
= \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|0\oplus f(x)\rangle|x\rangle =  
\frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|f(x)\rangle|x\rangle
$$
</li>
<p>
<li>Ahora aplicamos una H a cada cúbit en  $|x\rangle$ (recordemos que $H^{\otimes n}|x\rangle = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle)$<p>
$$
\begin{aligned}
    \lvert \psi_3 \rangle
          = \frac{1}{2^n} \sum_{x=0}^{2^n-1}|f(x)\rangle \sum_{i=0}^{2^n-1}(-1)^{x \cdot i} \vert i \rangle
          = \frac{1}{2^n}\sum_{x=0}^{2^n-1}
            \left[ \sum_{i=0}^{2^n-1}(-1)^{x \cdot i}
            \vert f(x) \rangle \vert i \rangle \right]
\end{aligned}
$$
</li><br>
</ol>

Sabemos que para cada $\vert x \rangle$ y para cada $\vert i \rangle$ se verifica que $\vert f(x) \rangle\vert i \rangle = \vert f(x\oplus c) \rangle\vert i \rangle$. Para un par de este tipo, los sumandos son:

$$
\frac{(-1)^{x \cdot i} + (-1)^{(x\oplus c) \cdot i}}{2^n}\vert f(x) \rangle\vert i \rangle =
\frac{(-1)^{x \cdot i} + (-1)^{(x\cdot i)\oplus (c\cdot i)}}{2^n}\vert f(x)\rangle\vert i\rangle =
\frac{(-1)^{x \cdot i}[1+(-1)^{c \cdot i}]}{2^n}\vert f(x)\rangle\vert i\rangle
$$

Así, si $c \cdot i = 1$ el termino correspondiente a $\vert f(x)\rangle\vert i\rangle$ se anula, y solo quedan los terminos para los que $c \cdot i = 0$.

Repitiendo varias veces la medida, podemos encontrar diferentes valores de $|i\rangle$ y, a partir de esas ecuaciones, obtener $c$.

**Ejemplo**

Supongamos $f:\{0,1\}^3 \rightarrow \{0,1\}^3$ tal que:
$$
f(0) = f(5) = 4\\
f(1) = f(4) = 1\\
f(2) = f(7) = 5\\
f(3) = f(6) = 7
$$

Es fácil comprobar que $f$ cumple los requisitos para aplicar el algoritmo de Simon.

Aplicando los pasos anteriores, en el paso 3 tenemos:

   
       
$$
\lvert \psi_3 \rangle
                 = \frac{1}{8}\sum_{x=0}^{7}
                    \sum_{i=0}^{7}(-1)^{x \cdot i}
                    \vert f(x)\rangle\vert i\rangle =
                    \frac{1}{8}[\\
|f(0)\rangle|0\rangle +  |f(0)\rangle|1\rangle +  |f(0)\rangle|2\rangle +  |f(0)\rangle|3\rangle + \\
|f(0)\rangle|4\rangle +  |f(0)\rangle|5\rangle +  |f(0)\rangle|6\rangle +  |f(0)\rangle|7\rangle +\\[5pt]
|f(1)\rangle|0\rangle -  |f(1)\rangle|1\rangle +  |f(1)\rangle|2\rangle -  |f(1)\rangle|3\rangle + \\
|f(1)\rangle|4\rangle -  |f(1)\rangle|5\rangle +  |f(1)\rangle|6\rangle -  |f(1)\rangle|7\rangle +\\[5pt]
|f(2)\rangle|0\rangle +  |f(2)\rangle|1\rangle -  |f(2)\rangle|2\rangle -  |f(2)\rangle|3\rangle + \\
|f(2)\rangle|4\rangle +  |f(2)\rangle|5\rangle -  |f(2)\rangle|6\rangle -  |f(2)\rangle|7\rangle +\\[5pt]
|f(3)\rangle|0\rangle -  |f(3)\rangle|1\rangle -  |f(3)\rangle|2\rangle +  |f(3)\rangle|3\rangle + \\
|f(3)\rangle|4\rangle -  |f(3)\rangle|5\rangle -  |f(3)\rangle|6\rangle +  |f(3)\rangle|7\rangle +\\[5pt]
|f(4)\rangle|0\rangle +  |f(4)\rangle|1\rangle +  |f(4)\rangle|2\rangle +  |f(4)\rangle|3\rangle - \\
|f(4)\rangle|4\rangle -  |f(4)\rangle|5\rangle -  |f(4)\rangle|6\rangle -  |f(4)\rangle|7\rangle +\\[5pt]
|f(5)\rangle|0\rangle -  |f(5)\rangle|1\rangle +  |f(5)\rangle|2\rangle -  |f(5)\rangle|3\rangle - \\
|f(5)\rangle|4\rangle +  |f(5)\rangle|5\rangle -  |f(5)\rangle|6\rangle +  |f(5)\rangle|7\rangle +\\[5pt]
|f(6)\rangle|0\rangle +  |f(6)\rangle|1\rangle -  |f(6)\rangle|2\rangle -  |f(6)\rangle|3\rangle - \\
|f(6)\rangle|4\rangle -  |f(6)\rangle|5\rangle +  |f(6)\rangle|6\rangle +  |f(6)\rangle|7\rangle +\\[5pt]
|f(7)\rangle|0\rangle -  |f(7)\rangle|1\rangle -  |f(7)\rangle|2\rangle +  |f(7)\rangle|3\rangle - \\
|f(7)\rangle|4\rangle +  |f(7)\rangle|5\rangle +  |f(7)\rangle|6\rangle -  |f(7)\rangle|7\rangle\ ]
$$

Substituyendo los valores de $f$:

$$
\lvert \psi_3 \rangle = \frac{1}{8}[\\
|4\rangle|0\rangle +  |4\rangle|1\rangle +  |4\rangle|2\rangle +  |4\rangle|3\rangle +  
|4\rangle|4\rangle +  |4\rangle|5\rangle +  |4\rangle|6\rangle +  |4\rangle|7\rangle +\\[5pt]
|1\rangle|0\rangle -  |1\rangle|1\rangle +  |1\rangle|2\rangle -  |1\rangle|3\rangle +  
|1\rangle|4\rangle -  |1\rangle|5\rangle +  |1\rangle|6\rangle -  |1\rangle|7\rangle +\\[5pt]
|5\rangle|0\rangle +  |5\rangle|1\rangle -  |5\rangle|2\rangle -  |5\rangle|3\rangle +  
|5\rangle|4\rangle +  |5\rangle|5\rangle -  |5\rangle|6\rangle -  |5\rangle|7\rangle +\\[5pt]
|7\rangle|0\rangle -  |7\rangle|1\rangle -  |7\rangle|2\rangle +  |7\rangle|3\rangle +  
|7\rangle|4\rangle -  |7\rangle|5\rangle -  |7\rangle|6\rangle +  |7\rangle|7\rangle +\\[5pt]
|1\rangle|0\rangle +  |1\rangle|1\rangle +  |1\rangle|2\rangle +  |1\rangle|3\rangle -  
|1\rangle|4\rangle -  |1\rangle|5\rangle -  |1\rangle|6\rangle -  |1\rangle|7\rangle +\\[5pt]
|4\rangle|0\rangle -  |4\rangle|1\rangle +  |4\rangle|2\rangle -  |4\rangle|3\rangle -  
|4\rangle|4\rangle +  |4\rangle|5\rangle -  |4\rangle|6\rangle +  |4\rangle|7\rangle +\\[5pt]
|7\rangle|0\rangle +  |7\rangle|1\rangle -  |7\rangle|2\rangle -  |7\rangle|3\rangle -  
|7\rangle|4\rangle -  |7\rangle|5\rangle +  |7\rangle|6\rangle +  |7\rangle|7\rangle +\\[5pt]
|5\rangle|0\rangle -  |5\rangle|1\rangle -  |5\rangle|2\rangle +  |5\rangle|3\rangle -  
|5\rangle|4\rangle +  |5\rangle|5\rangle +  |5\rangle|6\rangle -  |5\rangle|7\rangle\ ]
$$

Simplificando:
$$
\lvert \psi_3 \rangle = \frac{1}{4}\left[\\
|4\rangle|0\rangle+|1\rangle|0\rangle+|5\rangle|0\rangle+|7\rangle|0\rangle+\\
|4\rangle|2\rangle+|1\rangle|2\rangle-|5\rangle|2\rangle-|7\rangle|2\rangle+\\
|4\rangle|5\rangle-|1\rangle|5\rangle+|5\rangle|5\rangle-|7\rangle|5\rangle+\\
|4\rangle|7\rangle-|1\rangle|7\rangle-|5\rangle|7\rangle+|7\rangle|7\rangle\ \right]
$$

Que podemos escribir como:
$$
\lvert \psi_3 \rangle = \frac{1}{4}\left[\\
(|4\rangle+|1\rangle+|5\rangle+|7\rangle) \otimes|0\rangle+\\
(|4\rangle+|1\rangle-|5\rangle-|7\rangle)\otimes|2\rangle+\\
(|4\rangle-|1\rangle+|5\rangle-|7\rangle)\otimes|5\rangle+\\
(|4\rangle-|1\rangle-|5\rangle+|7\rangle)\otimes|7\rangle\ \right]
$$

Al medir los 3 bits $|x\rangle$ obtenemos, con igual probabilidad, $|000\rangle$, $|010\rangle$, $|101\rangle$ o $|111\rangle$. Para esos valores, su producto bit a bit por $c$ (módulo 2) da 0, por lo que tenemos, si $c=c_2c_1c_0$, con $c_i\in \{0,1\}$, las siguientes ecuaciones:

i. $|000\rangle\cdot c = 0\cdot c_2\oplus 0\cdot c_1 \oplus 0\cdot c_0 = 0$

ii. $|010\rangle\cdot c = 0\cdot c_2\oplus 1\cdot c_1 \oplus 0\cdot c_0 = 0$

iii. $|101\rangle\cdot c = 1\cdot c_2\oplus 0\cdot c_1 \oplus 1\cdot c_0 = 0$

iv. $|111\rangle\cdot c = 1\cdot c_2\oplus 1\cdot c_1 \oplus 1\cdot c_0 = 0$

 la ecuación (ii) indica que $c_1 = 0$, la (iii) que $c_2\oplus c_0 =0$ y la (iv) que $c_2\oplus c_1 \oplus c_0 =0$. Las únicas opciones para $c$ son $000$ o $101$, y como $c\neq 0$, tenemos que $c=5$.

### Ejercicio 6

Usamos el algoritmo de Simon para $n=3$ y obtenemos a la salida los siguientes estados: $|000\rangle$, $|001\rangle$, $|110\rangle$ y $|111\rangle$. ¿Cuánto vale $c$?

### Oráculo para el algoritmo de Simon

Supongamos $n=3$ y $c=110$. Una posible función es la siguiente:

$$
f(000) = f(110) = 000\\
f(001) = f(111) = 001\\
f(010) = f(100) = 100\\
f(011) = f(101) = 101
$$

Esta función se define como sigue. Sea $i$ la posición del bit a 1 menos significativo de $c$ (en el ejemplo, $i=1$). Definimos f como:

$$
f(x) =
\begin{cases}
x \text{, si } x_i = 0\\
x\oplus c \text{, si } x_i = 1
\end{cases}
$$

Para implementar esa función podemos hacer lo siguiente:

1. Copiamos el valor de $|x\rangle$ en los cúbits de ancilla para tener $|x\rangle|x\rangle$.
2. Sea $i$ la posición del bit a 1 menos significativo de $c$. Usamos puertas CNOT con:
   - Cúbit de control: $x_i$
   - Cúbits target: todas las ancillas con $j\ge i$ para las $j$ que verifiquen que $c_j = 1$.
3. Adicionalmente, se pueden permutar e invertir las ancillas para introducir variabilidad.

In [ ]:
# Ejemplo de oráculo para n=3 y c=110
# init es el valor inicial del registro x
def simon_oraculo_simple(init):
    x = QuantumRegister(3, name="|x\\rangle")
    ancilla = QuantumRegister(3, name="|ancilla\\rangle")
    meas = ClassicalRegister(3, name="Medida")
    qc = QuantumCircuit(x, ancilla, meas)
    qc.initialize(init)

    # Hacemos la copia de la x a las ancillas
    qc.cx(x,ancilla)

    qc.barrier()

    qc.cx(x[1],ancilla[1])
    qc.cx(x[1],ancilla[2])

    qc.barrier()

    qc.measure(ancilla, meas)
    return(qc)

In [ ]:
qc = simon_oraculo_simple(0)
qc.draw('mpl')

In [ ]:
# Probamos para diferentes valores
sim = AerSimulator(method='automatic')
for x in range(8):
    qc = simon_oraculo_simple(x)

    # Ejecutamos una simulación y obtenemos los resultados
    qc_obj = transpile(qc, sim)
    salida = sim.run(qc_obj, shots=1).result().get_counts()

    print('f({0:03b}) = {1}'.format(x,list(salida.keys())[0]))

### Ejercicio 7

Programa en Qiskit un oráculo para el algoritmo de Simon que implemente la función:$f:\{0,1\}^3 \rightarrow \{0,1\}^3$ tal que:

$$
f(0) = f(5) = 4\\
f(1) = f(4) = 5\\
f(2) = f(7) = 6\\
f(3) = f(6) = 7
$$

-------------------------------
--------------------------------
---------------------------------

# Algoritmo de búsqueda de Grover: amplificación de amplitud <a name="grover"></a>

Este algoritmo busca un dato en una lista no estructurada.

El oráculo $f:\{0,1\}^n \rightarrow \{0,1\}$ es tal que existe un único $x_i$ tal que:
$$
f(x) =
\begin{cases}
1, \text{si } x=x_i\\
0, \text{si } x\ne x_i
\end{cases}
$$

Buscamos $x_i$.

**Nota**: Es posible generalizar el algoritmo para cuando existe más de un dato $x_i$

**Solución clásica**

Si $N=2^n$ es el número de datos en la lista, un algoritmo clásico necesita efectuar, en el peor de los casos $N$ evaluaciones de $f(x)$, y de media $N/2$, por lo que la complejidad es $\mathcal{O}(2^n)$

**Solución cuántica**

El algoritmo cuántico ([L. K. Grover, 1996](http://doi.acm.org/10.1145/237814.237866)) solo necesita $\mathcal{O}\left(\sqrt{2^n}\right)$ evaluaciones.

El algoritmo tiene dos partes:

### Inversión de fase

Mediante el siguiente circuito invertimos la fase de $|x_i\rangle$:


<center><img src="https://drive.google.com/uc?export=view&id=1GQrOSYGXzA06IfD4LKAqwwe0V-B8NNFC" alt="Grover inversion de fase" width="800"  /></center>


Pasos:

<ol>
   <li>
        Estado inicial:
        

$$\vert \psi_0 \rangle =  \vert1\rangle\vert0\rangle^{\otimes n}$$


   </li>
    
   <li>
        Después de aplicar las primeras puertas H:
$$\vert \psi_1 \rangle = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} (|0\rangle-|1\rangle)\vert x\rangle $$
   </li>
   <p>
   <li>
        Aplicamos $U_f$ a $\vert y\rangle\vert x\rangle$ para obtener $\vert y \oplus f(x)\rangle\vert x\rangle$:
        $$
            \lvert \psi_2 \rangle  
                 = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(|0\oplus f(x)\rangle - |1\oplus f(x)\rangle)|x\rangle
        $$
          </li>
<p>
   <li>Como $f(x) = 1$ si $x = x_i$ y $f(x)=0$ en otro caso podemos escribir:
$$
\begin{aligned}
\lvert \psi_2 \rangle  
&= \frac{1}{\sqrt{2^{n+1}}}\left[\left(|0\oplus f(x_i)\rangle - |1\oplus f(x_i)\rangle\right)|x_i\rangle + \sum_{x\ne x_i}\left(|0\oplus f(x)\rangle - |1\oplus f(x)\rangle\right)|x\rangle\right] \\
&= \frac{1}{\sqrt{2^{n+1}}}\left[(|1\rangle - |0\rangle)|x_i\rangle + \sum_{x\ne x_i}\left(|0\rangle - |1\rangle\right)|x\rangle\right] \\
&= \frac{1}{\sqrt{2^{n+1}}}\left[\left(\sum_{x\ne x_i}\left(|0\rangle - |1\rangle\right)|x\rangle\right) - (|0\rangle - |1\rangle)|x_i\rangle\right] \\
&= \frac{|0\rangle - |1\rangle}{\sqrt{2}}\frac{\sum_{x\ne x_i}|x\rangle- |x_i\rangle}{\sqrt{2^{n}}}
\end{aligned}
$$  
          </li><br>
</ol>

El estado que medimos es, por lo tanto:

$$
\frac{\sum_{x\ne x_i}|x\rangle- |x_i\rangle}{\sqrt{2^{n}}}
$$

Vemos que el estado $|x_i\rangle$ aparece con signo negativo.

Implementar esta inversión de fase es simple.  Con 2 cúbits queremos invertir el estado $s = |10\rangle$:

1. Partimos de un estado en superposición
$$
\frac{|00\rangle + |01\rangle + |10\rangle + |11\rangle}{2}
$$
2. Invertimos los cúbits para los cuales hay un 0 en $s$ y obtenemos:
$$
\frac{|01\rangle + |00\rangle + |11\rangle + |10\rangle}{2}
$$
3. Aplicamos una puerta CZ entre los cúbits, que cambia el signo al estado $|11\rangle$
$$
\frac{|01\rangle + |00\rangle - |11\rangle + |10\rangle}{2}
$$
4. Volvemos a invertir los cúbits para los cuales hay un 0 en $s$:
$$
\frac{|00\rangle + |01\rangle - |10\rangle + |11\rangle}{2}
$$

El problema es que, al medir, obtenemos con igual probabilidad cualquier valor de $|x\rangle$ ya que lo único que diferencia a $|x_i\rangle$ es la fase. Necesitamos convertir diferencias de fase en diferencias de amplitud.

### Ejercicio 8

Programa en Qiskit un circuito de 3 cúbits que implemente la inversión de fase para un estado determinado, usando una puerta CCZ. Obten el vector de estado para ver el cambio de fase. Obtén también el histograma de las salidas para ver que no se distingue el estado marcado.

¿Es posible hacerlo para $n$ cúbits? Busca como implementar en Qiskit una puerta controlada por múltiples cúbits.

## Amplificación de amplitud

La AA se basa en aumentar la amplitud de los estados que tengan una fase negativa. La idea básica es la *inversión sobre la media*.

Supongamos el vector (decimal) $v=[10,10,10,-10,10]^T$. La media de esos valores es $\overline{v}=6$. Y la diferencia, en valor absoluto, entre $v[3]$ y el resto de valores es $0$.

Invertir sobre la media consiste en repetir esta operación:

$$
v^\prime = \overline{v} + (\overline{v}-v) = 2\overline{v} -v,\quad i = 0,1,2\ldots
$$

cambiando en cada iteración el signo del elemento negativo de $v$.

Aplicando esta operación al vector anterior obtenemos:

$$
v^\prime=2\times6 - v = [12,12,12,12,12]^T - [10,10,10,-10,10]^T = [2,2,2,22,2]^T
$$

La media sigue siendo la misma pero el tercer valor, antes negativo, se separa del resto. La diferencia ahora es $20$.

Podemos repetir el procedimiento, negando $v[3]$ para obtener $v=[2,2,2,-22,2]^T$ de media $\overline{v}=-2.8$. Volviendo a calcular $v^\prime$, obtenemos:

$$
v^\prime=2\times-2.8 - v = [-5.6,-5.6,-5.6,-5.6,-5.6]^T - [2,2,2,-22,2]^T = [-7.6,-7.6,-7.6,16.4,-7.6]^T
$$

La diferencia, en valor absoluto, entre $v[3]$ y el resto de valores se reduce a $8.8$.

En general, si $N$ es grande, a medida que aplicamos la inversión sobre la media esta diferencia se agranda, pero llega un punto en que empieza a disminuir.

Se puede demostrar que para un vector de $N$ elementos (con $N$ grande), el número de inversiones que obtienen la máxima separación es $\mathcal{O}(\sqrt{N})$.

Podemos expresar este mecanismo usando matrices. Sea, de nuevo, $v=[10,10,10,-10,10]^T$ y la matriz:

$$
A=\frac{1}{5}
\begin{bmatrix}
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

Está claro que $Av = [6,6,6,6,6]^T = [\overline{v}, \overline{v}, \overline{v}, \overline{v}, \overline{v}]^T$. La operación de inversión la podemos expresar como una operación matricial:

$$v^\prime=2Av-v=(2A-I)v$$

Para un vector de $2^n$ elementos, la matriz $2A-1$ será de $2^n\times 2^n$:
$$
2A-I=\frac{2}{2^n}
  \begin{bmatrix}
    1 & 1 & \dots & 1 \\
    1 & 1 & \dots & 1 \\
    \vdots & \vdots & \ddots & \vdots \\
    1 & 1 & \dots & 1
  \end{bmatrix}-
  \begin{bmatrix}
    1 & 0 & \dots & 0 \\
    0 & 1 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & 1
  \end{bmatrix} =\frac{1}{2^{n-1}}
  \begin{bmatrix}
    1-2^{n-1} & 1 & \dots & 1 \\
    1 & 1-2^{n-1} & \dots & 1 \\
    \vdots & \vdots & \ddots & \vdots \\
    1 & 1 & \dots & 1-2^{n-1}
  \end{bmatrix}
$$

Se puede demostrar que $U = 2A-I$ es una matriz unitaria, por lo que se puede contruir en un circuito cuántico.


Para el caso de 2 cúbits, la matriz $2A-I$ queda:
$$
2A-I=\frac{1}{2}
  \begin{bmatrix}
    -1 & 1 & 1 & 1 \\
    1 & -1 & 1 & 1 \\
    1 & 1 & -1 & 1 \\
    1 & 1 & 1 & -1
  \end{bmatrix}
$$

Esta matriz se llama operador de difusión o _difusor_ $R_s$.

Es fácil verificar que si $|s\rangle = |+\rangle^{\otimes n}$ es el estado de superposición completa, entonces $A = |s\rangle\langle s|$, por lo que:

$$
R_s = 2A-I = 2|s\rangle\langle s| -I
$$

#### Implementación de $R_s$

Dado que $|s\rangle = |+\rangle^{\otimes n}= H^{\otimes n}|0\rangle^{\otimes n}$ se tiene :

$$
R_s = 2\left(H^{\otimes n}|0\rangle^{\otimes n}\langle 0|^{\otimes n}H^{\dagger\otimes n}\right)-I = 2\left(H^{\otimes n}|0\rangle^{\otimes n}\langle 0|^{\otimes n}H^{\otimes n}\right)-I
$$

ya que $H^\dagger = H$.

Además, podemos escribir $I = H^{\otimes n}H^{\otimes n}$. Por lo tanto:

$$
R_s = 2\left(H^{\otimes n}|0\rangle^{\otimes n}\langle 0|^{\otimes n}H^{\otimes n}\right)- H^{\otimes n}H^{\otimes n} = H^{\otimes n}\left(2|0\rangle^{\otimes n}\langle 0|^{\otimes n}-I\right)H^{\otimes n} = H^{\otimes n}R_0H^{\otimes n}
$$

siendo:

$$
R_0 = 2|0\rangle^{\otimes n}\langle 0|^{\otimes n}-I
$$

Es decir, podemos implementar $R_s$ como $R_0$ rodeado de puertas H:

<center><img src="https://drive.google.com/uc?export=view&id=1OrYgEeJuhO9RAVpvd1BoS2H-nayFKKhz" alt="Difusor Rs" width="300"  /></center>


#### Implementación de $R_0$

Es posible comprobar que $R_0$ aplicado a cualquier estado cambia el signo de $|i\rangle$ para todo $i\ne 0$.

Ejemplo:

$$
R_0|++\rangle = \frac{1}{2}\left(2|00\rangle\langle 00|-I\right)\left(|00\rangle+|01\rangle+|10\rangle+|11\rangle\right) = \\
\frac{1}{2}\left(2|00\rangle\langle 00|00\rangle + 2|00\rangle\langle 00|01\rangle + 2|00\rangle\langle 00|10\rangle + 2|00\rangle\langle 00|11\rangle - |00\rangle-|01\rangle-|10\rangle-|11\rangle\right) = \frac{1}{2}\left(|00\rangle-|01\rangle-|10\rangle-|11\rangle\right)
$$

Para implementar esta operación, podemos empezar usando una puerta Z controlada:

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1xpp0U6OmMiNaMaobqtMPIyINGyKGbtdg" alt="Difusor R0: Z controlada" width="150"  /></center>
<br>

Si se aplica al estado $|+\rangle^{\otimes n}$ cambia el signo del estado $|1\rangle^{\otimes n}$:

$$
\frac{1}{2}\left(|00\rangle+|01\rangle+|10\rangle-|11\rangle\right)
$$

Si añadimos puertas X a todos los cúbits, cambiamos el signo del estado $|0\rangle^{\otimes n}$

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1k3s4JvVpfzR74-OsvteU6MrbK3P50AvZ" alt="Difusor R0: puertas X" width="200"  /></center>
<br>

$$
\frac{1}{2}\left(-|00\rangle+|01\rangle+|10\rangle+|11\rangle\right)
$$

Para obtener el resultado que buscamos, podemos usar que las puertas ZXZX aplicadas a un cúbit le cambián el signo:

$$
ZXZX|\phi\rangle = -|\phi\rangle
$$

Si se aplican estas puertas al cúbit más significativo, obtendremos el resultado:

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1kER6ai8Qd0G5fijpk3cDvUAccDGLrrfH" alt="Difusor R0: puertas ZXZX" width="400"  /></center>
<br>


$$
\frac{1}{2}\left(|00\rangle-|01\rangle-|10\rangle-|11\rangle\right)
$$

Se pueden simplificar las dos X consecutivas, por lo que la implementación de $R_0$ queda:


<br>
<center><img src="https://drive.google.com/uc?export=view&id=1kowaV2C_ykZ1xyxqR7bVoHmrND2m0SFt" alt="Difusor R0: simplificado" width="400"  /></center>
<br>


### Ejercicio 9

Programa en Qiskit una función que devuelva una puerta $R_s$ para un número arbitrario de cúbits.

Obtén la matriz unitaria asociada y comprueba que coincide con lo esperado.

Juntando la inversión de fase con la amplificación de amplitud, el circuito que implementa el algoritmo de Grover queda:

<!--img src="https://github.com/tarabelo/PIAC-apuntes/blob/main/images/Grover2.svg?raw=1" alt="Grover completo" style="width: 800px;"/-->

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1Jgx29CcPQelYV3WdFDMzHElXbmxeFHNH" alt="Difusor R0: simplificado" width="800"  /></center>
<br>


## Interpretación geométrica del algoritmo de Grover

Sea $|s\rangle$ el estado inicial del algoritmo de Grover (superposición completa) y sea $w$ el valor para el que $f(w)=1$. Podemos escribir el estado inicial como:

$$
|s\rangle = \frac{1}{\sqrt{2^{n}}}{\sum_{x=0}^{2^n-1}|x\rangle} = \frac{\sum_{x\ne w}|x\rangle + |w\rangle}{\sqrt{2^{n}}} = \sqrt{\frac{2^n-1}{2^n}}\frac{1}{\sqrt{2^n-1}}\sum_{x\ne w}|x\rangle + \frac{1}{\sqrt{2^n}}|w\rangle
$$

Si denotamos:
$$
\begin{aligned}
|r\rangle &= \frac{1}{\sqrt{2^n-1}}\sum_{x\ne x_i}|x\rangle\\
N &= 2^n
\end{aligned}
$$

podemos escribir este estado como:

$$
|s\rangle = \frac{1}{\sqrt{N}}|w\rangle + \sqrt{\frac{N-1}{N}}|r\rangle
$$

Y como

$$
\left(\frac{1}{\sqrt{N}}\right)^2 + \left(\sqrt{\frac{N-1}{N}}\right)^2 = 1
$$

existe $\theta$ tal que podemos escribir el estado como:

$$
|s\rangle =\sin\theta|w\rangle + \cos\theta|r\rangle
$$

Podemos representar este estado así:

<!--img src="https://github.com/tarabelo/PIAC-apuntes/blob/main/images/GeoGrover1.png?raw=1" alt="Geo Grover 1" style="width: 200px;"/-->

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1LTxkItpwErgsBT4-nRrwjON4AbQkgR4K" alt="Geo Grover 1" width="300"  /></center>
<br>


Si a este estado le aplicamos la inversión de fase, $U_f$, se le cambia el signo a $|w\rangle$:

$$
U_f|s\rangle = -\sin\theta|w\rangle + \cos\theta|r\rangle
$$

que equivale a una reflexión con respecto al eje $|r\rangle$:

<!--img src="https://github.com/tarabelo/PIAC-apuntes/blob/main/images/GeoGrover2.png?raw=1" alt="Geo Grover 2" style="width: 200px;"/-->

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1fGJs667obKVpQ-fXI8ZZSJAPL3PqHn-R" alt="Geo Grover 2" width="300"  /></center>
<br>


Ahora aplicamos la amplificación de amplitud. El difusor $R_s = 2|s\rangle\langle s| -I$ mantiene el estado $|s\rangle$ pero invierte cualquier estado ortogonal a $|s\rangle$:

$$
\begin{aligned}
R_s|s\rangle &= 2|s\rangle\langle s|s\rangle - |s\rangle = 2|s\rangle - |s\rangle = |s\rangle\\
R_s|s^\perp\rangle &= 2|s\rangle\langle s|s^\perp\rangle - |s^\perp\rangle = -|s^\perp\rangle\\
\end{aligned}
$$

ya que $\langle s|s\rangle=1$ y $\langle s|s^\perp\rangle=0$. Es decir, $R_s$ produce una reflexión sobre $|s\rangle$.

Si aplicamos $R_s$ al estado $U_f|s\rangle$ tendremos el estado reflejado sobre $|s\rangle$:

<!--img src="https://github.com/tarabelo/PIAC-apuntes/blob/main/images/GeoGrover3.png?raw=1" alt="Geo Grover 3" style="width: 200px;"/-->

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1nP_8pEpK2DNPMEF-5QFpaJJqrI0wscW3" alt="Geo Grover 3" width="300"  /></center>
<br>

Si aplicamos de nuevo las dos reflexiones $U_f$ y $R_s$ tendremos:

<!--img src="https://github.com/tarabelo/PIAC-apuntes/blob/main/images/GeoGrover4.png?raw=1" alt="Geo Grover 4" style="width: 200px;"/-->

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1lvjeZmrnYT7eSPFr2YpuQLEKZTy_rKsI" alt="Geo Grover 4" width="300"  /></center>
<br>

Si continuamos aplicando las reflexiones, el estado se irá acercando a $|w\rangle$ que es la solución buscada.

**¿Cuándo se alcanza $|w\rangle$?**

Cada vez que aplicamos las reflexiones, el estado avanza un ángulo $2\theta$ con respecto a $|s\rangle$. Sea $t$ el número de reflexiones para llegar a $|w\rangle$:

$$
\begin{aligned}
\theta + 2\theta\cdot t = \frac{\pi}{2} \Rightarrow t = \frac{\pi}{4\theta} - \frac{1}{2}
\end{aligned}
$$

Como $\sin\theta = \frac{1}{\sqrt{N}}$, si $N$ es grande:

$$
\theta = \sin^{-1}\left(\frac{1}{\sqrt{N}}\right) \approx \frac{1}{\sqrt{N}}
$$

Por lo que:

$$
t \approx \frac{\pi}{4}\sqrt{N} - \frac{1}{2} \approx \frac{\pi}{4}\sqrt{N}
$$

Por lo que el número de repeticiones del algoritmo es $\mathcal{O}(\sqrt{2^n})$